# Using Census Reporter API to Geographically Target Demographics
## Kenneth R. Miller
#### We have a coffee company and are looking to expand to other geographic areas. From looking at a Mintel report on premium coffee, we've decided to target hispanic individuals 18-34 years of age, who have an income from 50k to 74.9k dollars. Using CBSA (geographic ID codes) and Census Reporter, we are able to see exactly how many individuals meet our demographic target per metropolitan statistical area.

In [12]:
import pandas as pd
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import json
import requests
import time 

# Data can be found in github repository
# This contains CBSA codes, name of the statistical area, and a 2017 population estimate for that area
msadf = pd.read_csv("msas.csv")

# Indexing the data frame by the CBSA instead of typical index codes
msadf = msadf.set_index("CBSA", drop = False)

#### Identifying the tables we want and using geographic areas to generate request URLs

In [13]:
# The specific table IDs we want...age, racial identity and income
tableids = ['B01001', 'B03002', 'B19001']
# Request URLs require single strings, not lists
tableidstring = ','.join(tableids)

# This list will contain all the request URLs in the same order as the CBSAs
urllst = []
# Initializing a variable to contain the geographic IDs
geoid = 0  
# This list will contain all the CBSAs in the format that censusreporter needs
geoidlst = []

# Iterating over each index and row in the dataframe
    # In this case, the index IS the CBSA value because we reindexed the data frame
for index, row in msadf.iterrows():
    # Geoid is just the is just the index with 31000US on the front
    geoid = '31000US' + str(index) 
    # Creating a request url for each statistical area
    requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring, geoid)
    urllst.append(requesturl) # Appending the urls to a list of urls
    # Appending a geoid to a list of geoids
    geoidlst.append(str(geoid))

msadf['URL'] = urllst
msadf['GEO_ID'] = geoidlst

#### Getting counts from Census Reporter of each of our demographics by geographic location

In [ ]:
# Creating 3 empty lists, each to contain counts of our selected variables
age = []
race = []
income = []
# There are lots of codes needed to get an age range from 18-34, so I made a list that I can iterate through
age_get = ['B01001007', 'B01001008', 'B01001009', 'B01001010', 'B01001011', 'B01001012']

# Iterating over each URL in our URL list
for element in urllst:
    # Use the current URL to get the json
    loadedjson = requests.get(element)
    loadedjson.json()
    parsedjson = json.loads(loadedjson.text)
    # This is the CBSA of the current URL, just taken off the end
    locstr = element[-12:]
    
    # Creating three temporary data frames of the data we need using the location gleaned from the URL
    agedf = pd.DataFrame.from_dict(parsedjson['data'][locstr]['B01001'])
    racedf = pd.DataFrame.from_dict(parsedjson['data'][locstr]['B03002'])
    incomedf = pd.DataFrame.from_dict(parsedjson['data'][locstr]['B19001'])
    
    # Iterating over that list of age codes and adding it to a sum
    age_sum = 0
    for amt in age_get:
        age_sum = agedf['estimate'][amt] + age_sum
    
    # Appending the sum to the age list
    age.append(age_sum)
    # Appending the income data to the income list
    income.append(incomedf['estimate']['B19001011'] + incomedf['estimate']['B19001012'])
    # Appending the race data to the race list (hispanics only)
    race.append(racedf['estimate']['B03002012'])
    
    # Slows down the for loop so the api requests don't bog down the website
    # This waits the remainder of the second that the for loop takes each time.
    time.sleep(-time.time()%1)

#### Putting these counts into our data frame

In [ ]:
# Adding the three lists as columns in our data frame
# This works because the URL list is in the same order as the index of msadf 
    # and these lists are in that order as well
msadf['Age Count'] = age
msadf['Income Count'] = income
msadf['Hispanic Count'] = race

# Calculating Percentage of population
msadf['Age Perc.'] = msadf['Age Count']/msadf['POPESTIMATE2017']

msadf['Income Perc.'] = msadf['Income Count']/msadf['POPESTIMATE2017']

msadf['Race Perc.'] = msadf['Hispanic Count']/msadf['POPESTIMATE2017']

msadf['Average'] = (msadf['Race Perc.'] + msadf['Income Perc.'] + msadf['Age Perc.'])/3


#### Answering questions with our retreived data

In [ ]:
# Question 1: What city has highest number of individuals in our income range?
msadf['NAME'][msadf['Income Count'] == max(msadf['Income Count'])]
# NY

# Question 2: What city has highest count of 18-34 year olds?   
msadf['NAME'][msadf['Age Count'] == max(msadf['Age Count'])]
# NY

# Question 3: What city has highest count of hispanics?
msadf['NAME'][msadf['Hispanic Count'] == max(msadf['Hispanic Count'])]
# LA
